In [2]:
#import libraries
from mordred import Calculator, descriptors, is_missing
from sklearn.externals.joblib import load
import pandas as pd
import math
from sklearn.linear_model import LinearRegression
from pandas import DataFrame
from rdkit.Chem.inchi import *
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
from rdkit.Chem.rdMolDescriptors import GetMorganFingerprintAsBitVect
from rdkit.DataStructs.cDataStructs import ConvertToNumpyArray
import rdkit
# from tqdm import tqdm
from tqdm.notebook import tqdm
import xgboost as xgb
from pandas import Panel
tqdm.pandas()

C:\Users\Sergey\anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
C:\Users\Sergey\anaconda3\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
C:\Users\Sergey\anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
# function for descriptors using MORDERED
calc = Calculator(descriptors, ignore_3D=True)
def mord(mol, nBits=1613, errors_as_zeros=True):
    try:
        result = calc(mol)
        desc_list = [r if not is_missing(r) else 0 for r in result]
        np_arr = np.array(desc_list)
        return np_arr
    except:
        return np.NaN if not errors_as_zeros else np.zeros((nBits,), dtype=np.float32)

In [4]:
#Load standard_scaler and pre-trained models
s=load('std_scaler.bin')
mord_bin= xgb.Booster()
mord_reg = xgb.Booster()
mord_bin.load_model('mordred_bin.model')
mord_reg.load_model('mordred_reg.model')

[16:24:29] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:727: Loading model from XGBoost < 1.0.0, consider saving it again for improved compatibility
[16:24:29] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:727: Loading model from XGBoost < 1.0.0, consider saving it again for improved compatibility


C:\Users\Sergey\anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.22.2.post1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [29]:
#Load data
data = pd.read_csv('./data/evaluation_set.csv', delimiter=";")


In [30]:
mord_reg.save_model('mord_reg.model')
mord_reg.load_model('mord_reg.model')

In [32]:
data

,Name,rt,InChI,Formula,rt_mord,rt_morg
0,Phenibut,19.96,InChI=1S/C10H13NO2/c11-7-9(6-10(12)13)8-4-2-1-...,C10H13NO2,20.647002,22.641521
1,Captopril,21.66,InChI=1S/C9H15NO3S/c1-6(5-14)8(11)10-4-2-3-7(1...,C9H15NO3S,22.891241,24.166239
2,Furazolidone,23.30,InChI=1S/C8H7N3O5/c12-8-10(3-4-15-8)9-5-6-1-2-...,C8H7N3O5,25.496308,27.648764
3,Naproxen,36.96,InChI=1S/C14H14O3/c1-9(14(15)16)10-3-4-12-8-13...,C14H14O3,31.657609,34.193724
4,Tolperisone,23.82,InChI=1S/C16H23NO/c1-13-6-8-15(9-7-13)16(18)14...,C16H23NO,26.016688,24.762681
5,Oxymetazoline,25.99,"InChI=1S/C16H24N2O/c1-10-8-13(16(3,4)5)15(19)1...",C16H24N2O,24.634514,27.357964
6,Fenoterol,18.67,InChI=1S/C17H21NO4/c1-11(6-12-2-4-14(19)5-3-12...,C17H21NO4,19.875086,19.453912
7,Ipratropium,20.98,InChI=1S/C20H30NO3/c1-14(2)21(3)16-9-10-17(21)...,C20H30NO3+,30.167533,26.271259
8,Clemastine,32.12,"InChI=1S/C21H26ClNO/c1-21(17-7-4-3-5-8-17,18-1...",C21H26ClNO,30.133525,35.472999
9,Ofloxacin,21.66,InChI=1S/C18H20FN3O4/c1-10-9-26-17-14-11(16(23...,C18H20FN3O4,21.011344,20.225572


In [33]:
# Converting to appropriate format:
data['mol'] = data['InChI'].progress_apply(MolFromInchi)
data['mordred'] = data['mol'].progress_apply(mord)
sample = xgb.DMatrix(np.vstack(data['mordred'].values))

In [34]:
pred_bin = mord_bin.predict(sample)
pred_reg = mord_reg.predict(sample)
pred_reg= s.inverse_transform(pred_reg).reshape(-1,1)

In [35]:
pred_reg

array([[ 552.23444],
       [ 609.4966 ],
       [ 675.96533],
       [ 833.172  ],
       [ 689.2429 ],
       [ 653.9765 ],
       [ 532.5389 ],
       [ 795.15247],
       [ 794.2847 ],
       [ 561.5307 ],
       [1024.644  ],
       [1009.25836],
       [ 848.83325],
       [ 523.4007 ],
       [ 679.55084],
       [ 707.6106 ],
       [ 719.96783],
       [1012.386  ],
       [ 599.0274 ],
       [ 635.75525]], dtype=float32)

In [40]:
data['Binary_retained'] = pred_bin
data['Predicted_RT'] = pred_reg

out = pd.DataFrame(list(zip(data['Name'], data['InChI'],data['Binary_retained'], data['Predicted_RT'])), columns=['CID', 'InChI', 'Binary_retained', 'Predicted_RT'])
export_csv = out.to_csv ('output_C21H27NO_methadone.csv', index = None, header=True, sep=';')

In [41]:
data

,Name,rt,InChI,Formula,rt_mord,rt_morg,mol,mordred,Binary_retained,Predicted_RT
0,Phenibut,19.96,InChI=1S/C10H13NO2/c11-7-9(6-10(12)13)8-4-2-1-...,C10H13NO2,20.647002,22.641521,<rdkit.Chem.rdchem.Mol object at 0x00000142543...,"[9.370727640387596, 8.882635714303776, 1.0, 1....",0.628860,552.234436
1,Captopril,21.66,InChI=1S/C9H15NO3S/c1-6(5-14)8(11)10-4-2-3-7(1...,C9H15NO3S,22.891241,24.166239,<rdkit.Chem.rdchem.Mol object at 0x00000142543...,"[10.175293677496859, 10.514528190831122, 1.0, ...",0.999999,609.496582
2,Furazolidone,23.30,InChI=1S/C8H7N3O5/c12-8-10(3-4-15-8)9-5-6-1-2-...,C8H7N3O5,25.496308,27.648764,<rdkit.Chem.rdchem.Mol object at 0x00000142543...,"[12.268104450355086, 11.292849378205043, 0.0, ...",0.998867,675.965332
3,Naproxen,36.96,InChI=1S/C14H14O3/c1-9(14(15)16)10-3-4-12-8-13...,C14H14O3,31.657609,34.193724,<rdkit.Chem.rdchem.Mol object at 0x00000142543...,"[12.93477111702175, 10.892176440070545, 1.0, 0...",1.000000,833.171997
4,Tolperisone,23.82,InChI=1S/C16H23NO/c1-13-6-8-15(9-7-13)16(18)14...,C16H23NO,26.016688,24.762681,<rdkit.Chem.rdchem.Mol object at 0x00000142543...,"[13.68231801272818, 11.52280474429733, 0.0, 1....",1.000000,689.242920
5,Oxymetazoline,25.99,"InChI=1S/C16H24N2O/c1-10-8-13(16(3,4)5)15(19)1...",C16H24N2O,24.634514,27.357964,<rdkit.Chem.rdchem.Mol object at 0x00000142543...,"[14.723690875849991, 13.48575674277234, 0.0, 2...",0.999827,653.976501
6,Fenoterol,18.67,InChI=1S/C17H21NO4/c1-11(6-12-2-4-14(19)5-3-12...,C17H21NO4,19.875086,19.453912,<rdkit.Chem.rdchem.Mol object at 0x00000142543...,"[16.769964851476608, 13.624750062054396, 0.0, ...",0.888655,532.538879
7,Ipratropium,20.98,InChI=1S/C20H30NO3/c1-14(2)21(3)16-9-10-17(21)...,C20H30NO3+,30.167533,26.271259,<rdkit.Chem.rdchem.Mol object at 0x00000142543...,"[18.60615543317597, 15.962822650584902, 0.0, 1...",1.000000,795.152466
8,Clemastine,32.12,"InChI=1S/C21H26ClNO/c1-21(17-7-4-3-5-8-17,18-1...",C21H26ClNO,30.133525,35.472999,<rdkit.Chem.rdchem.Mol object at 0x00000142543...,"[18.598815304773318, 15.627015732825276, 0.0, ...",0.999997,794.284729
9,Ofloxacin,21.66,InChI=1S/C18H20FN3O4/c1-10-9-26-17-14-11(16(23...,C18H20FN3O4,21.011344,20.225572,<rdkit.Chem.rdchem.Mol object at 0x00000142543...,"[20.758034307658143, 16.164168990233684, 1.0, ...",0.999998,561.530701


In [ ]:
data['ae'] = abs(data.)